# AutoSkLearn Benchmark: HTGR Micro-Core Quadrant Power

**Input**

- `theta1`: Angle of control drum in quadrant 1 (radians) 
- `theta2`: Angle of control drum in quadrant 1 (radians) 
- `theta3`: Angle of control drum in quadrant 2 (radians)  
- `theta4`: Angle of control drum in quadrant 2 (radians)
- `theta5`: Angle of control drum in quadrant 3 (radians)
- `theta6`: Angle of control drum in quadrant 3 (radians)
- `theta7`: Angle of control drum in quadrant 4 (radians)  
- `theta8`: Angle of control drum in quadrant 4 (radians)  

**Output** 

- `fluxQ1` : Neutron flux in quadrant 1 ($\frac{neutrons}{cm^{2} s}$)
- `fluxQ2` : Neutron flux in quadrant 2 ($\frac{neutrons}{cm^{2} s}$)
- `fluxQ3` : Neutron flux in quadrant 3 ($\frac{neutrons}{cm^{2} s}$)
- `fluxQ4` : Neutron flux in quadrant 4 ($\frac{neutrons}{cm^{2} s}$)


We will be benchmarking the complete HTGR dataset of 3004 samples using H2O ML (version 3.46.0.5) in efforts to compare pyMAISE to other industry standard ML benchmarking frameworks. We will be following the same procedures we did in the original HTGR example, first extending the dataset to 3004 samples using symmetry, and then training and evaluating to compare results.

In [1]:
# Importing Packages
import time
import numpy as np
import pandas as pd

# Set display option to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Set the width of the columns
pd.set_option('display.width', None)

# See the full content of each column
pd.set_option('display.max_colwidth', None)

import xarray as xr
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, MinMaxScaler
# Plot settings
matplotlib_settings = {
    "font.size": 12,
    "legend.fontsize": 11,
    "figure.figsize": (8, 8)
}
plt.rcParams.update(**matplotlib_settings)

## Processing the data

First, we will load the raw data into a dataframe and print it out.

In [2]:
import os

cwd = os.getcwd()
new_cwd = cwd.replace("/docs/source/benchmarks", "/pyMAISE/datasets")

# Define the full path to the microreactor.csv file
csv_path = os.path.join(new_cwd, 'microreactor.csv')

# Load the CSV file into a pandas DataFrame
raw_data = pd.read_csv(csv_path)
raw_data.head()

,sample number,cpu_time,runtime,k,fluxQ1,fluxQ2,fluxQ3,fluxQ4,k_uncert,flux_runcertQ1,flux_runcertQ2,flux_runcertQ3,flux_runcertQ4,fissQ1,fissQ2,fissQ3,fissQ4,fissEQ1,fissEQ2,fissEQ3,fissEQ4,fiss_runcertQ1,fiss_runcertQ2,fiss_runcertQ3,fiss_runcertQ4,fissE_runcertQ1,fissE_runcertQ2,fissE_runcertQ3,fissE_runcertQ4,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8
0,sample_00000,4260.0,200.0,0.998328,2.580000e+19,2.590000e+19,2.670000e+19,2.560000e+19,0.00019,0.00112,0.00111,0.00111,0.00108,8.490000e+16,8.490000e+16,8.480000e+16,8.490000e+16,2751290,2751060,2749270,2750450,0.00060,0.00060,0.00063,0.00062,0.00060,0.00060,0.00063,0.00062,5.919526,2.369503,2.923656,4.488987,3.683212,4.008905,4.970368,2.987966
1,sample_00001,2570.0,130.0,0.988522,2.550000e+19,2.530000e+19,2.510000e+19,2.510000e+19,0.00025,0.00142,0.00148,0.00154,0.00150,8.490000e+16,8.490000e+16,8.490000e+16,8.490000e+16,2750610,2750210,2750150,2750110,0.00076,0.00077,0.00084,0.00074,0.00076,0.00077,0.00084,0.00074,2.162380,0.273624,0.927741,4.595586,2.598824,0.170167,2.124048,4.980209
2,sample_00002,2590.0,130.0,1.004610,2.570000e+19,2.580000e+19,2.520000e+19,2.520000e+19,0.00025,0.00167,0.00163,0.00161,0.00165,8.480000e+16,8.480000e+16,8.490000e+16,8.490000e+16,2748870,2749690,2752250,2751840,0.00076,0.00077,0.00086,0.00080,0.00076,0.00077,0.00086,0.00080,0.450100,0.006301,2.512217,3.313864,1.913458,3.582252,0.280764,4.888595
3,sample_00003,2580.0,129.0,0.991892,2.570000e+19,2.580000e+19,2.520000e+19,2.560000e+19,0.00025,0.00197,0.00193,0.00195,0.00200,8.480000e+16,8.490000e+16,8.480000e+16,8.470000e+16,2748920,2750720,2749330,2746220,0.00082,0.00076,0.00080,0.00078,0.00082,0.00076,0.00080,0.00078,0.461105,4.825628,3.771356,2.599278,2.056019,0.007332,1.106786,5.504671
4,sample_00004,2570.0,129.0,0.985047,2.540000e+19,2.620000e+19,2.580000e+19,2.520000e+19,0.00025,0.00167,0.00167,0.00172,0.00169,8.480000e+16,8.490000e+16,8.480000e+16,8.490000e+16,2748910,2753130,2747870,2752420,0.00080,0.00081,0.00082,0.00083,0.00080,0.00081,0.00082,0.00083,5.248202,3.549416,3.333632,3.907310,2.095312,5.585145,3.774253,2.480120


We are then going to create input and output dataframes by defining our input and output variables.

In [3]:
# Create the input DataFrame with theta values
input_columns = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8']
inputs = raw_data[input_columns]

# Create the output DataFrame with flux values
output_columns = ['fluxQ1', 'fluxQ2', 'fluxQ3', 'fluxQ4']
outputs = raw_data[output_columns]

Below, we print out the results for input and output then also create a combined dataset with both.

In [4]:
inputs.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8
0,5.919526,2.369503,2.923656,4.488987,3.683212,4.008905,4.970368,2.987966
1,2.162380,0.273624,0.927741,4.595586,2.598824,0.170167,2.124048,4.980209
2,0.450100,0.006301,2.512217,3.313864,1.913458,3.582252,0.280764,4.888595
3,0.461105,4.825628,3.771356,2.599278,2.056019,0.007332,1.106786,5.504671
4,5.248202,3.549416,3.333632,3.907310,2.095312,5.585145,3.774253,2.480120


In [5]:
outputs.head()

,fluxQ1,fluxQ2,fluxQ3,fluxQ4
0,2.580000e+19,2.590000e+19,2.670000e+19,2.560000e+19
1,2.550000e+19,2.530000e+19,2.510000e+19,2.510000e+19
2,2.570000e+19,2.580000e+19,2.520000e+19,2.520000e+19
3,2.570000e+19,2.580000e+19,2.520000e+19,2.560000e+19
4,2.540000e+19,2.620000e+19,2.580000e+19,2.520000e+19


In [6]:
combined_df = pd.concat([inputs, outputs], axis=1)
print(combined_df.head())

     theta1    theta2    theta3    theta4    theta5    theta6    theta7  \
0  5.919526  2.369503  2.923656  4.488987  3.683212  4.008905  4.970368   
1  2.162380  0.273624  0.927741  4.595586  2.598824  0.170167  2.124048   
2  0.450100  0.006301  2.512217  3.313864  1.913458  3.582252  0.280764   
3  0.461105  4.825628  3.771356  2.599278  2.056019  0.007332  1.106786   
4  5.248202  3.549416  3.333632  3.907310  2.095312  5.585145  3.774253   

     theta8        fluxQ1        fluxQ2        fluxQ3        fluxQ4  
0  2.987966  2.580000e+19  2.590000e+19  2.670000e+19  2.560000e+19  
1  4.980209  2.550000e+19  2.530000e+19  2.510000e+19  2.510000e+19  
2  4.888595  2.570000e+19  2.580000e+19  2.520000e+19  2.520000e+19  
3  5.504671  2.570000e+19  2.580000e+19  2.520000e+19  2.560000e+19  
4  2.480120  2.540000e+19  2.620000e+19  2.580000e+19  2.520000e+19  


Now it is time to extend the dataset to 3004 samples. This is done in the same way as in the original HTGR, replicating the same steps below.

In [7]:
# Credit to mult_sym and g21 from https://github.com/deanrp2/MicroControl/blob/main/pmdata/utils.py#L51
theta_cols = [f"theta{i + 1}" for i in range(8)]
flux_cols = [f"fluxQ{i + 1}" for i in range(4)]

def mult_samples(data):
    # Create empty arrays
    ht = xr.DataArray(
        np.zeros(data.shape), 
        coords={
            "index": [f"{idx}_h" for idx in data.coords["index"].values],
            "variable": data.coords["variable"],
        },
    )
    vt = xr.DataArray(
        np.zeros(data.shape), 
        coords={
            "index": [f"{idx}_v" for idx in data.coords["index"].values],
            "variable": data.coords["variable"],
        },
    )
    rt = xr.DataArray(
        np.zeros(data.shape),     
        coords={
            "index": [f"{idx}_r" for idx in data.coords["index"].values],
            "variable": data.coords["variable"],
        },
    )

    # Swap drum positions
    hkey = [f"theta{i}" for i in np.array([3, 2, 1, 0, 7, 6, 5, 4], dtype=int) + 1]
    vkey = [f"theta{i}" for i in np.array([7, 6, 5, 4, 3, 2, 1, 0], dtype=int) + 1]
    rkey = [f"theta{i}" for i in np.array([4, 5, 6, 7, 0, 1, 2, 3], dtype=int) + 1]

    ht.loc[:, hkey] = data.loc[:, theta_cols].values
    vt.loc[:, vkey] = data.loc[:, theta_cols].values
    rt.loc[:, rkey] = data.loc[:, theta_cols].values

    # Adjust angles
    ht.loc[:, hkey] = (3 * np.pi - ht.loc[:, hkey].loc[:, hkey]) % (2 * np.pi)
    vt.loc[:, vkey] = (2 * np.pi - vt.loc[:, hkey].loc[:, vkey]) % (2 * np.pi)
    rt.loc[:, rkey] = (np.pi + rt.loc[:, hkey].loc[:, rkey]) % (2 * np.pi)

    # Fill quadrant tallies
    hkey = [2, 1, 4, 3]
    vkey = [4, 3, 2, 1]
    rkey = [3, 4, 1, 2]

    ht.loc[:, [f"fluxQ{i}" for i in hkey]] = data.loc[:, flux_cols].values
    vt.loc[:, [f"fluxQ{i}" for i in vkey]] = data.loc[:, flux_cols].values
    rt.loc[:, [f"fluxQ{i}" for i in rkey]] = data.loc[:, flux_cols].values

    sym_data = xr.concat([data, ht, vt, rt], dim="index").sortby("index")
    
    # Normalize fluxes
    sym_data.loc[:, flux_cols].values = Normalizer().transform(sym_data.loc[:, flux_cols].values)
    
    # Convert global coordinate system to local
    loc_offsets = np.array(
        [3.6820187359906447, 4.067668586955522, 2.2155167202240653 - np.pi, 2.6011665711889425 - np.pi, 
         0.5404260824008517, 0.9260759333657285, 5.3571093738138575 - np.pi, 5.742759224778734 - np.pi]
    )

    # Apply correct 0 point
    sym_data.loc[:, theta_cols] = sym_data.loc[:, theta_cols] - loc_offsets + 2 * np.pi

    # Reverse necessary angles
    sym_data.loc[:, [f"theta{i}" for i in [3,4,5,6]]] *= -1

    # Scale all to [0, 2 * np.pi]
    sym_data.loc[:, theta_cols] = sym_data.loc[:, theta_cols] % (2 * np.pi)
        
    return sym_data

In [8]:
train_data, test_data = train_test_split(combined_df, test_size=0.3)

# Convert to xarray DataArray and specify the index as a coordinate
train_data_xr = xr.DataArray(
    train_data.values,
    coords={"index": train_data.index, "variable": train_data.columns},
    dims=["index", "variable"]
)
test_data_xr = xr.DataArray(
    test_data.values,
    coords={"index": test_data.index, "variable": test_data.columns},
    dims=["index", "variable"]
)

In [9]:
sym_train_data = mult_samples(train_data_xr)
sym_test_data = mult_samples(test_data_xr)
print(f"Multiplied training shape: {sym_train_data.shape}, Multiplied testing shape: {sym_test_data.shape}")

Multiplied training shape: (2100, 12), Multiplied testing shape: (904, 12)


As seen above, we end up with data the same size as the original HTGR. Below, we are going to Min-Max the X_data and normalize the y_data.

In [10]:
# Min-Max scaling data 
def scale_data(train_data, test_data, scaler):
    train_data.values = scaler.fit_transform(
        train_data.values.reshape(-1, train_data.shape[-1])
    ).reshape(train_data.shape)
    test_data.values = scaler.transform(
        test_data.values.reshape(-1, test_data.shape[-1])
    ).reshape(test_data.shape)
    
    # Return data
    return train_data, test_data, scaler

xtrain_arr, xtest_arr , _ = scale_data(sym_train_data.loc[:, theta_cols], sym_test_data.loc[:, theta_cols], MinMaxScaler())
ytrain_arr, ytest_arr, _ = scale_data(sym_train_data.loc[:, flux_cols], sym_test_data.loc[:, flux_cols], Normalizer(norm="l1"))

In [11]:
xtrain = xtrain_arr.to_pandas()
xtest = xtest_arr.to_pandas()
ytrain = ytrain_arr.to_pandas()
ytest = ytest_arr.to_pandas()

## Benchmark with Auto-Sklearn

Now that preprocessing is done, it is time to use the Auto-Sklearn framework. We are going to first import the necessary libraries.

In [12]:
#Import things required from Autosklearn
from autosklearn.regression import AutoSklearnRegressor
from sklearn.multioutput import MultiOutputRegressor
import autosklearn.metrics as metrics

We are going to set all the settings for auto-sklearn. Since pyMAISE took around 3 hours to complete parameter finding, we are going to cap it around the same. We will set ensemble_size=0 and ensemble_class=None since auto-sklearn automatically tries to create ensembles while we want single estimators. All available cores will be used (n_jobs=-1) and we also set cross-validation on with 5 folds as done in the orignal HTGR. Finally, we will only allow auto-sklearn to pick from decision trees, random forest, and k nearest neighbors, because these are what we used in the original HTGR notebook.

In [13]:
auto_reg = AutoSklearnRegressor(
    n_jobs=-1,                         # Number of parallel jobs to run
    initial_configurations_via_metalearning=0,  # Start from scratch
    resampling_strategy='cv',         # Cross-validation
    resampling_strategy_arguments={'folds': 5},  # Set number of folds for cross-validation
    memory_limit=None,  # No memory limit 
    time_left_for_this_task= (3*60*60), #3 hrs
    ensemble_class=None, #Disable ensemble building or use SingleBest to obtain only use the single best model instead of an ensemble
    ensemble_kwargs={
      "ensemble_size": 0,  
    },
    include={
        'regressor': [
            "decision_tree",  # Decision Tree Regressor
            "random_forest",  # Random Forest Regressor
            "k_nearest_neighbors"  # K-Nearest Neighbors Regressor
        ],
    },
    metric= metrics.r2,
)

We then run the parameter selection using .fit() on the training dataset. 

In [14]:
auto_reg.fit(xtrain, ytrain)

[WARNING] [2024-10-28 12:08:03,691:Client-AutoML(1):a6f6378d-9546-11ef-895b-08bfb876ce9f] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (10799.725003)
[WARNING] [2024-10-28 12:08:03,691:Client-AutoML(1):a6f6378d-9546-11ef-895b-08bfb876ce9f] Capping the per_run_time_limit to 5399.0 to have time for a least 2 models in each process.
RunKey(config_id=1, instance_id='{"task_id": "a6f6378d-9546-11ef-895b-08bfb876ce9f"}', seed=0, budget=0.0) RunValue(cost=0.2900568491999885, time=19.91672372817993, status=<StatusType.SUCCESS: 1>, starttime=1730131684.1462064, endtime=1730131704.0797384, additional_info={'duration': 19.056434631347656, 'num_run': 2, 'train_loss': 0.04029638749909272, 'configuration_origin': 'Default'})
RunKey(config_id=2, instance_id='{"task_id": "a6f6378d-9546-11ef-895b-08bfb876ce9f"}', seed=0, budget=0.0) RunValue(cost=0.8384699802929064, time=2.1806697845458984, status=<StatusType.SUCCESS: 1>

AutoSklearnRegressor(ensemble_class=None, ensemble_kwargs={'ensemble_size': 0},
                     include={'regressor': ['decision_tree', 'random_forest',
                                            'k_nearest_neighbors']},
                     initial_configurations_via_metalearning=0,
                     memory_limit=None, metric=r2, n_jobs=-1,
                     per_run_time_limit=34560, resampling_strategy='cv',
                     resampling_strategy_arguments={'folds': 5},
                     time_left_for_this_task=10800)

We then refit (actually training this time) on the entire dataset since the previous was using cross validation to pick the best hyperparameters.

In [15]:
auto_reg.refit(xtrain, ytrain)

AutoSklearnRegressor(ensemble_class=None, ensemble_kwargs={'ensemble_size': 0},
                     include={'regressor': ['decision_tree', 'random_forest',
                                            'k_nearest_neighbors']},
                     initial_configurations_via_metalearning=0,
                     memory_limit=None, metric=r2, n_jobs=-1,
                     per_run_time_limit=34560, resampling_strategy='cv',
                     resampling_strategy_arguments={'folds': 5},
                     time_left_for_this_task=10800)

Now that training is done, we can show the best models below for this task. As shown, we can see that auto_sklearn just chose random forest models for this task, possibly because they performed the best on this dataset.

In [33]:
print(auto_reg.leaderboard())

          rank  ensemble_weight           type      cost   duration
model_id                                                           
598          1              1.0  random_forest  0.152436  102.20542


We can show all the different models and their rankings below, showing each of their configurations. Note that auto-sklearn returns a pipeline that actually preprocesses the data first. This pipeline has 3 preprocessing steps: Imputation (not needed here), Scaling (already completed), and Polynomial Degree fitting. To keep our benchmark consistent with the pyMAISE example, we will take the hyperparameters learnt here and make a new pipeline in the next section without imputation and scaling.

In [32]:
#print(auto_reg.show_models())
auto_reg.get_models_with_weights()

[(1.0,
  SimpleRegressionPipeline({'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'polynomial', 'regressor:__choice__': 'random_forest', 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'standardize', 'feature_preprocessor:polynomial:degree': 2, 'feature_preprocessor:polynomial:include_bias': 'False', 'feature_preprocessor:polynomial:interaction_only': 'False', 'regressor:random_forest:bootstrap': 'True', 'regressor:random_forest:criterion': 'mse', 'regressor:random_forest:max_depth': 'None', 'regressor:random_forest:max_features': 0.9654893761894044, 'regressor:random_forest:max_leaf_nodes': 'None', 'regressor:random_forest:min_impurity_decrease': 0.0, 'regressor:random_forest:min_samples_leaf': 1, 'regressor:random_forest:min_samples_split': 2, 'regressor:random_forest:min_weight_fraction_leaf': 0.0},
  dataset_properties={
    '

Finally it is time to benchmark the results for HTGR. We will rebuild the pipeline with the learned hyperparameters as stated above to align with the same methodolgy as in the pyMAISE example. Below, we print out the best results for R2, MAE, RMSE, and MAPE. We can see that the results for pyMAISE and auto-sklearn are almost identical at around 0.70 $R^2$. The other metrics also line up similarly.

In [65]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import numpy as np

# Define the pipeline with all preprocessing steps and the model
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)),  # Polynomial features
    ('regressor', RandomForestRegressor(
        bootstrap=True,
        criterion='mse',
        max_depth=None,
        max_features=0.9655,  # Close to 0.9654893761894044
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        min_samples_leaf=1,
        min_samples_split=2,
        min_weight_fraction_leaf=0.0,
        random_state=42
    ))
])

# Fit the pipeline on the training data
pipeline.fit(xtrain, ytrain)

# Make predictions
predictions = pipeline.predict(xtest)

# Evaluate the model
print("R2 score:", r2_score(ytest, predictions))
print("MAE score:", mean_absolute_error(ytest, predictions))
print("RMSE score:", np.sqrt(mean_squared_error(ytest, predictions)))
print("MAPE score:", mean_absolute_percentage_error(ytest, predictions))

R2 score: 0.7053476042225316
MAE score: 0.001872428722826578
RMSE score: 0.002374621226524507
MAPE score: 0.007475683139485854


We can also optionally print out the configuration space below which shows all the hyperparameter spaces which we trained out. This gives us an idea for what decisions Auto-Sklearn made when running this example. 

In [19]:
print(auto_reg.get_configuration_space(xtrain, ytrain))

Configuration space object:
  Hyperparameters:
    data_preprocessor:__choice__, Type: Categorical, Choices: {feature_type}, Default: feature_type
    data_preprocessor:feature_type:numerical_transformer:imputation:strategy, Type: Categorical, Choices: {mean, median, most_frequent}, Default: mean
    data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__, Type: Categorical, Choices: {minmax, none, normalize, power_transformer, quantile_transformer, robust_scaler, standardize}, Default: standardize
    data_preprocessor:feature_type:numerical_transformer:rescaling:quantile_transformer:n_quantiles, Type: UniformInteger, Range: [10, 2000], Default: 1000
    data_preprocessor:feature_type:numerical_transformer:rescaling:quantile_transformer:output_distribution, Type: Categorical, Choices: {normal, uniform}, Default: normal
    data_preprocessor:feature_type:numerical_transformer:rescaling:robust_scaler:q_max, Type: UniformFloat, Range: [0.7, 0.999], Default: 0.75
    dat